# Решение задач классификации.

* Порешаем теоретические задачи на логистическую регрессию.
* Научимся решать линейные и нелинейные задачи классификации с логистической регрессии.
* Разберём, как подбирать гиперпараметры моделей по сетке.
* Потренируемся применять пайплайны.

## Решение теоретических задач

**Задание 1**. Рассмотрим логистическую регрессию в задаче предсказания целевой переменной по двум признакам: $a(x)=\sigma(w_0+w_1\cdot x_1 +w_2\cdot x_2)$. 

После оценки качества алгоритма по кросс-валидации выяснилось, что модель переобучилась. Какие из нижеперечисленных подходов корректно описаны и их можно предпринять для уменьшения переобучения?

1) Уберём константный коэффициент $w_0$, так как он увеличивает сложность модели и при этом не влияет на обобщающую способность модели

2) Добавим к модели регуляризатор $w_0^2 +w_1^2 +w_2^2$, так как l2-регуляризация может уменьшить переобучение

3) Добавим к модели регуляризатор вида $|w_1|+|w_2|$, так как l1-регуляризация может уменьшить переобучение

4) Добавим к модели регуляризатор вида $[w_1\neq 0]+[w_2\neq 0]$, так как l0-регуляризация может уменьшить переобучение (здесь [x] = 1, если выражение x верно, а иначе 0)

5) Добавим полиномиальных признаков второй степени, чтобы увеличить обобщающую способность модели


**Задание 2**. ![Задание 5](задача2.png) 

##  Задача 1: многоклассовая классификация цветков ириса.

![Ирисы](irises.jpg)

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LogisticRegression

import warnings
warnings.filterwarnings('ignore')

In [ ]:
from sklearn.datasets import load_iris

iris = load_iris()

X = iris.data
y = iris.target

X.shape, y.shape

Выведите на экран все различные значения целевой переменной, а также распределение классов

In [ ]:
#your code here

In [ ]:
from collections import Counter

Counter(y)

Для решения этой задачи будем использовать **линейную модель - логистическую регрессию**.

Разбейте данные на train и test (test - 20% от всех данных).

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

Xtrain, Xtest, ytrain, ytest = ...

Обучите логистическую регрессию на train и выведем качество (accuracy) на train и на test.

In [ ]:
#your code here

Посмотрите на качество алгоритма на кросс-валидации.

In [ ]:
from sklearn.model_selection import cross_val_score

#your code here

Получили очень высокую точность модели.

Посмотрим, можно ли улучшить качество модели **подбором её гиперпараметров**. Логистическая регрессия имеет гиперпараметр C - подберём его. 

* Подбор гиперпараметров традиционно осуществляется "по сетке" с помощью функции ***GridSearchCV***. 

GridSearchCV - это класс, который ищет с помощью кросс-валидации (CV) по сетке наилучшую комбинацию искомых параметров. 

*За что отвечает гиперпараметр С?*

In [ ]:
from sklearn.model_selection import GridSearchCV

params = {'C': np.arange(0.1,1.1,0.1)}
grid = GridSearchCV(model, params)

grid.fit(X, y)

print('best score:', grid.best_score_)
print('best params:', grid.best_params_)

Задача уже решена очень хорошо. На этом можно ставить точку.

## Задача 2: задача XOR.

![How it works](XOR.png)

Создадим данные для задачи XOR

In [ ]:
rng = np.random.RandomState(0)
X = rng.randn(200, 2)
y = np.logical_xor(X[:, 0] > 0, X[:, 1] > 0)

plt.scatter(X[:, 0], X[:, 1], s=30, c=y, cmap=plt.cm.Paired);

Будем использовать функцию, которая рисует разделяющую границу, проведенную классификатором.

In [ ]:
def plot_boundary(clf, X, y, plot_title):
    xx, yy = np.meshgrid(np.linspace(-3, 3, 50), np.linspace(-3, 3, 50))
    clf.fit(X, y)
    # plot the decision function for each datapoint on the grid
    Z = clf.predict_proba(np.vstack((xx.ravel(), yy.ravel())).T)[:, 1]
    Z = Z.reshape(xx.shape)

    image = plt.imshow(Z, interpolation='nearest',
                       extent=(xx.min(), xx.max(), yy.min(), yy.max()),
                       aspect='auto', origin='lower', cmap=plt.cm.PuOr_r)
    contours = plt.contour(xx, yy, Z, levels=[0], linewidths=2,
                               linetypes='--')
    plt.scatter(X[:, 0], X[:, 1], s=30, c=y, cmap=plt.cm.Paired)
    plt.xticks(())
    plt.yticks(())
    plt.xlabel(r'$x_1$')
    plt.ylabel(r'$x_2$')
    plt.axis([-3, 3, -3, 3])
    plt.colorbar(image)
    plt.title(plot_title, fontsize=12);

In [ ]:
plot_boundary(LogisticRegression(solver='lbfgs'), X, y, "Logistic Regression, XOR problem")

Оценим качество полученной классификации.

In [ ]:
cross_val_score(model, X, y, cv=3, scoring='accuracy').mean()

Мы видим, что задача решена плохо. Попробуем подобрать гиперпараметр C в логистической регрессии.

In [ ]:
params = {'C': np.arange(0.1,1.1,0.1)}
grid = GridSearchCV(model, params)

grid.fit(X, y)

print('best score:', grid.best_score_)
print('best params:', grid.best_params_)

Видим, что подбор гиперпараметра не помог, и задача всё ещё решена плохо. Получается, что линейный классификатор (а именно, логистическая регрессия), плохо решает эту задачу. Это и понятно, ведь *данные нелинейные*.

Попробуем добавить в качестве признаков полиномиальные признаки степени 2.

Теперь у нас будут признаки не только $(1, x_1, x_2)$, но и $1, x_1, x_2, x_1^2, x_1x_2, x_2^2$.

Будем использовать удобную функцию *Pipeline* для описания модели.

***Pipeline***.
* С помощью Pipeline можно соорудить модель, в которой кроме непосредственно применения некоторого алгоритма происходит какая-либо обработка данных.
* В нашей задаче необходимо сначала извлечь полиномиальные признаки, а затем обучить логистическую регрессию. Эти два действия можно осуществить внутри одной "комбинированной" модели с помощью Pipeline.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

logit_pipe = Pipeline([('poly', PolynomialFeatures(degree=2)),
                       ('logit', LogisticRegression())])

Найдите оптимальное значение гиперпараметра C с помощью GridSearchCV и оцените качество модели (на кросс-валидации).

In [ ]:
#your code here

In [ ]:
plot_boundary(logit_pipe, X, y,
              "Logistic Regression + quadratic features. XOR problem")

* **Таким образом, при добавлении новых нелинейных признаков с помощью линейного классификатора можно решить линейно неразделимую задачу.**

* **Другой способ решить такую задачу - сразу использовать нелинейные классификаторы (попробуем на следующих семинарах).** 